In [1]:
# Install dependencies - não é necessário correr
import sys
!{sys.executable} -m pip install hdfs

In [2]:
from os import PathLike
from hdfs import InsecureClient
from pyspark.sql import SparkSession
from pyspark.sql import Row
from delta import *
from pyspark.sql.types import LongType, StringType, StructField, StructType, BooleanType, ArrayType, IntegerType, FloatType, DoubleType
from pyspark.sql.functions import when, col, concat, lit, substring, avg, sum

client = InsecureClient("http://hdfs-nn:9870", user="anonymous")

# warehouse_location points to the default location for managed databases and tables
warehouse_location = 'hdfs://hdfs-nn:9000/projeto/fisico/gold/'

builder = SparkSession \
    .builder \
    .master("local[2]") \
    .appName("Python Spark DataFrames and SQL") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("hive.metastore.uris", "thrift://hive-metastore:9083") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:1.0.0") \
    .enableHiveSupport() \

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/spark-3.2.0-bin-hadoop3.2/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jovyan/.ivy2/cache
The jars for the packages stored in: /home/jovyan/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-0b28c1cc-4a16-44e6-b8be-84b60b8ac207;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 375ms :: artifacts dl 17ms
	:: modules in use:
	io.delta#delta-core_2.12;1.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      

In [3]:
# apagar tudo do hdfs

to_path = "/projeto/fisico/gold/utilization/"

client.delete(to_path,True)

False

In [4]:
# read data from the silver tables
df = spark.table("silver.stats")

In [5]:
df.printSchema()
df.count()

root
 |-- Year: string (nullable = true)
 |-- Player: string (nullable = true)
 |-- Pos: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tm: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- G: integer (nullable = true)
 |-- GS: integer (nullable = true)
 |-- Wins: integer (nullable = true)
 |-- Losses: integer (nullable = true)
 |-- WP: double (nullable = true)
 |-- MP: integer (nullable = true)
 |-- PER: double (nullable = true)
 |-- TSP: double (nullable = true)
 |-- 3PAr: double (nullable = true)
 |-- FTr: double (nullable = true)
 |-- ORBP: double (nullable = true)
 |-- DRBP: double (nullable = true)
 |-- TRBP: double (nullable = true)
 |-- ASTP: double (nullable = true)
 |-- STLP: double (nullable = true)
 |-- BLKP: double (nullable = true)
 |-- TOVP: double (nullable = true)
 |-- USGP: double (nullable = true)
 |-- OWS: double (nullable = true)
 |-- DWS: double (nullable = true)
 |-- WS: double (nullable = true)
 |-- WS48: double (nullable = tru

25475

In [6]:
# apagar todos os dados onde não há info de wins e losses
gd_df = df.filter("Year > 1946") \
          .filter("Year < 2018")

In [7]:
gd_df = gd_df.withColumn("Year", gd_df["Year"].cast(IntegerType()))

In [8]:
gd_df.printSchema()
#gd_df.count()
#gd_df.toPandas()

root
 |-- Year: integer (nullable = true)
 |-- Player: string (nullable = true)
 |-- Pos: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tm: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- G: integer (nullable = true)
 |-- GS: integer (nullable = true)
 |-- Wins: integer (nullable = true)
 |-- Losses: integer (nullable = true)
 |-- WP: double (nullable = true)
 |-- MP: integer (nullable = true)
 |-- PER: double (nullable = true)
 |-- TSP: double (nullable = true)
 |-- 3PAr: double (nullable = true)
 |-- FTr: double (nullable = true)
 |-- ORBP: double (nullable = true)
 |-- DRBP: double (nullable = true)
 |-- TRBP: double (nullable = true)
 |-- ASTP: double (nullable = true)
 |-- STLP: double (nullable = true)
 |-- BLKP: double (nullable = true)
 |-- TOVP: double (nullable = true)
 |-- USGP: double (nullable = true)
 |-- OWS: double (nullable = true)
 |-- DWS: double (nullable = true)
 |-- WS: double (nullable = true)
 |-- WS48: double (nullable = tr

In [9]:
# ficar apenas com colunas necessarias
gd_df = gd_df.select("Year","Player","Pos","Tm","Team","G","GS","MP","Wins","Losses","WP")

In [10]:
# write to delta table
gd_df \
    .write \
    .format("delta") \
    .mode("overwrite") \
    .save("hdfs://hdfs-nn:9000//projeto/fisico/gold/utilization/")

22/01/14 23:23:49 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [11]:
# check the results in the table
spark.table("gold.utilization").show()
spark.table("gold.utilization").count()

+----+-----------------+---+---+--------------------+---+---+----+----+------+-----+
|Year|           Player|Pos| Tm|                Team|  G| GS|  MP|Wins|Losses|   WP|
+----+-----------------+---+---+--------------------+---+---+----+----+------+-----+
|2015|       Quincy Acy| PF|NYK|     New York Knicks| 68| 22|1287|  17|    65|0.207|
|2015|     Jordan Adams| SG|MEM|   Memphis Grizzlies| 30|  0| 248|  55|    27|0.671|
|2015|     Steven Adams|  C|OKC|Oklahoma City Thu...| 70| 67|1771|  45|    37|0.549|
|2015|      Jeff Adrien| PF|MIN|Minnesota Timberw...| 17|  0| 215|  16|    66|0.195|
|2015|    Arron Afflalo| SG|TOT|                null| 78| 72|2502|null|  null| null|
|2015|    Arron Afflalo| SG|DEN|      Denver Nuggets| 53| 53|1750|  30|    52|0.366|
|2015|    Arron Afflalo| SG|POR|Portland Trail Bl...| 25| 19| 752|  51|    31|0.622|
|2015|    Alexis Ajinca|  C|NOP|New Orleans Pelicans| 68|  8| 957|  45|    37|0.549|
|2015|   Furkan Aldemir| PF|PHI|  Philadelphia 76ers| 41|  9| 540

24624